In [1]:
import pyspark
from pyspark import SparkContext

## Spark Context

In [2]:
sc = SparkContext()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/26 23:44:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
nums = sc.parallelize([1, 2, 3, 4, 5])
nums.take(1)

[1]

In [4]:
squared = nums.map(lambda x: x**x).collect()
for num in squared:
    print(num)

1
4
27
256
3125


## SQLContext

In [5]:
from pyspark.sql import SQLContext, Row

In [6]:
sqlContext = SQLContext(sc)

/opt/homebrew/lib/python3.10/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [9]:
lst = [('John',19),('Smith',29),('Adam',35),('Henry',50)]
rdd = sc.parallelize(lst)
ppl = rdd.map(lambda x: Row(name=x[0], age=int(x[1])))
df = sqlContext.createDataFrame(ppl)

In [10]:
df.printSchema()
df.show()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)

+-----+---+
| name|age|
+-----+---+
| John| 19|
|Smith| 29|
| Adam| 35|
|Henry| 50|
+-----+---+

